In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import csv

In [60]:
def get_data(comm_markets_path, distances_path):
    '''
    A function that pulls in the data from marketcommodities.ini and the flc dump file and scrapes it. 
    +++++++++
    Parameters:
    comm_markets_path (str): the path of market_commodities.ini
    distances_path (str): the path of the flc dump file
    +++++++++
    Returns
    distances (Dataframe): pandas dataframe of all the viable distances between bases
    bases(list of dictonaries): list containing all of the base entries in market_commodities.ini in dictonary 
    form. Dictonaies hold base_code, base_name, and list of commodities traded. 
    comm_set (set): set of all commodities bought or sold on all bases
    base_names (list): all base names
    '''


    distances = pd.read_csv(distances_path, names = ['start', 'end', 'time'])
    distances = distances[distances['start']!=distances['end']]
    distances = distances[distances['time']!=-1]

    with open(comm_markets_path, 'r') as file:
        lines = file.readlines() 
    
    comment_count = 0
    bases = []
    commodities =[]
    for i in lines[2:]:
        if i.lower() == '[basegood]\n' or i == ';EVERYTHING BELOW THIS LINE IS DATABASE.':
            if comment =='':
                comment = base_code
            bases.append({'base_code':base_code.strip('\n'), 'Name':comment.strip('\n'), 'commodities':commodities})
            commodities=[]
            comment = ''
            comment_count = 0
        if i[:4].lower() == "base":
            base_code = i[7:]
            
        if i[:1] == ';' and comment_count == 0:
            comment = i[1:]
            comment_count+=1
        elif i[:1]==';':
            print(i)
            comment +'\n'+ i
        if i[:10].lower() == 'marketgood':
            t =i[13:].split()
            l = [float(j.strip(',')) for j in t[1:6]]
            l.append(float(t.pop()))
            l.append(t[0])
            commodities.append(l)
    
    comm_set = set([])
    for i in bases: #this loop in loop adds all the commodities to a set so we know how many commodities to display in our applet
        j = i['commodities']
        for k in j:
            comm_set.add(k[6])
    base_names=[(base['Name'],base['base_code']) for base in bases]
    
    return distances, bases, comm_set, base_names,

In [4]:
def which_sell(commodity_list):
    '''
    A helperfunction that digs through the bases list generated by get_data and finds which ones sell any of a given list of commodities
    
    Paramaters
    commodity_list(list): a list of the commodity codes (strings) you want to find where they're sold
    
    Returns
    list of bases (list): list of base codes that sell any of the given commodities
    '''
    sells_set = set([])
    for i in bases:
        for k in i['commodities']:
            if k[-1] in commodity_list and k[-3]== 0:
                sells_set.add(i['base_code'])
    l = list(sells_set)
    return [i.strip('\n') for i in l]

In [5]:
def sort_by_closest_base(list_of_bases, distances):
    '''
    a function that takes the distances dataframe and any list of bases and sorts all the other bases out into the lists of bases closest to those bases in the list of bases. 
    Its got a lot of fuckery, and maybe can be given another pass for better algorithmic complexity. I'm so sorry. 
    
    Parameters 
    list_of_bases (list): a list of string base codes
    distances (DataFrame): the base to base travel times from flcomp formatted as a DataFrame by get_data()
    
    Returns:
    sorted_l (list of lists) list of list of bases, sorted in the same shape as the list of bases fed in, 
    with each list containing those bases that are closest to the base listed at the same index in list_of_bases
    '''
    k = [distances[distances['start'] == i] for i in list_of_bases] # makes a list of dataframes of routes starting from the bases in question
    k = [i[~i['end'].isin(list_of_bases)].reset_index() for i in k] # gets rid of routes between bases in the  list (because you don't want one producer to sell to the other)
    
    df = pd.DataFrame()
    for idx, i in enumerate(list_of_bases): #peices the list k back together into a larger thing
        df[i]=k[idx]['time']
    df['base'] = k[0]['end']
    df = df.set_index('base')
    #print(df)
    df['time'] = df.min(axis = 1)
    df['shortest'] = df.idxmin(axis = 1)
    df.reset_index(inplace = True)
    df['output'] = list(zip(df['time'], df['base']))

    sorted_bases = df[['base','time','shortest','output']]
    
    #sorted_bases = sorted_bases.reset_index()
    sorted_l =[]

    for i in list_of_bases:
        sorted_l.append(sorted_bases[sorted_bases['shortest']==i]['output'].to_list())

    return sorted_l

In [6]:
def commodities_from_config(config):
    '''
    takes the config (whatever that turns out to be like, right now its a list of lists) and spits out all of the commodites used
    '''
    
    comm_set = set()
    for locations in config:
        comm_set.update(locations[1])
        comm_set.update(locations[2])
        comm_set.update(locations[3])
    return comm_set

In [7]:
def lookup(bases):
    '''
    A helperfunction to produce a pair of dictonaries that associate base codes to base names and back again. 
    
    '''
    keys = [base['Name'].strip('\n') for base in bases]
    values = [base['base_code'].strip('\n').lower() for base in bases]
    base_code_lookup = {keys[i]: values[i] for i in range(len(keys))} 
    base_name_lookup = {values[i]:keys[i] for i in range(len(keys))}
    return base_code_lookup, base_name_lookup

In [8]:
def find_nearest_x(base_code, num, distances):
    '''
    given the list of distances, finds the nearest num bases to base given
    '''
    output = distances[distances['start'] == base_code].sort_values('time', axis = 0).head(num)['end'].to_list()
    return output

In [9]:
def gen_market_from_config(config, distances):
    '''
    function that reads the config file(currently a list of lists) and turns that into a dictonary much closer to the marketgoods ini for the backwards-parser to turn into a text file. 
    A COUPLE BASES AREN'T GETTING THEIR FRICKING PURCHASES RECORDED STILL. SORT THAT OUT. 
    
    Parameters
    ++++++++
    config (list of lists): bases that buy and sell each commodity, formatted into a list containing [base name, commodity base produces, comody base resells, comodity base consumes]
    distances(df): dataframe of all base to base travel times
    ++++++++
    Returns
    market_goods(defaultdict(list)):
    '''
    
    comms = commodities_from_config(config)
    
    
    #loop that rolls through the config file and sorts out the bases that consume and produce each thing. 
    market = []
    for commodity in comms:
        bases_that_consume = []
        bases_that_produce = []
            
        for base in config:
            if commodity in base[1]:
                bases_that_produce.append(base[0])
            if commodity in base[3]:
                bases_that_consume.append(base[0])
        
        market.append({'commodity': commodity, 'produces':bases_that_produce,'consumes':bases_that_consume})
    
    
    #loop that takes the market produced above and turns it into a dictonary of bases, commodities, buy-sells, and travel ties. 
    market_goods = defaultdict(list)
    for commodity in market:
    #print(market_goods)
        specific_distances = distances[distances['start'].isin(commodity['produces'])]
        
   
        for location in commodity['produces']:
            market_goods[location].append((commodity['commodity'], 'sells', 1 )) #sets distance for sellers to 1 for purposes of multiplying price by 1. 
        
        sorts = sort_by_closest_base(commodity['produces'], specific_distances)
        #print(sorts)
        consumer = commodity['consumes']
        
        filter_sorts = [[base for base in group if base[1] in consumer] for group in sorts] #nasty comprehension in a comprehension filters out all the bases that are not set as consumers of the commodity
        
        for group in filter_sorts:
            for location in group:
                market_goods[location[1]].append((commodity['commodity'], 'buy', location[0]))
                
                
    return market_goods


In [10]:
def write_ini(file_path, market, config):
    '''
    function that takes the market dictonary and formats it for freelancer, and saves it as a text file
    ++++++++++
    Parameters
    file_path (str): name and path of file
    market (defaultdict(list)): per base formatting of all commodities
    ++++++++++
    Returns:
    none, 
    writes a file
    '''
    lines = []
    for item in market.keys():

        lines.append('[BaseGood]')
        lines.append('base = '+ item)
        lines.append(';'+base_name_lookup[item])

        for commodity in market[item]:
            if commodity[1] == 'sells':
                function_tag = '1'
            else:
                function_tag = '0'
            price_mult = str(commodity[2]) # For a working version, this needs to reference something in config to set prices
            mgood =['Marketgood =',commodity[0],'0, -1, 0, 0,', function_tag, str(commodity[2])]
            lines.append(' '.join(mgood))

    with open(file_path, 'w') as f:
        for line in lines:
            f.write(line+'\n')


In [58]:
path_markets = 'market_commodities.ini'
distances_path = 'dump.csv'

In [61]:
distances, bases, comm_set, base_names = get_data(path_markets, distances_path)
base_code_lookup, base_name_lookup = lookup(bases)

In [62]:
bases[1]

{'base_code': 'Br01_02_base',
 'Name': 'Southampton Shipyard',
 'commodities': [[0.0, -1.0, 0.0, 0.0, 1.0, 1.104, 'commodity_ablativearmor,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.31, 'commodity_aluminium,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.0, 'commodity_ambassadors,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.0, 'commodity_arbiters,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.45, 'commodity_basic_alloys,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.21, 'commodity_beryllium,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.25, 'commodity_beryllium_ca,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.26, 'commodity_bioprocessors,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.096, 'commodity_bluediamonds,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.05, 'commodity_bluejillies,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.05, 'commodity_boron,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.17, 'commodity_cobalt,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.77, 'commodity_construction_machinery,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 3.12, 'commodity_consumer_goods_br,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.17, 'commodity_co

In [13]:
path = 'Discovery_Economy/Bases.csv'
def load_xoria_base_info(path):
    
    xoria_bases = pd.read_csv(path)
    return xoria_bases

In [14]:
path = 'Discovery_Economy/Bases.csv'
xbases = load_xoria_base_info(path)


In [80]:
def to_string(obj):
    return obj.values[0]
xoria_bases = pd.read_csv(path)
xoria_bases = xoria_bases.fillna('NaN')

TypeError: applymap() got an unexpected keyword argument 'columns'

In [21]:
bases[1]['base_code'].strip('\n')

'Br01_02_base'

In [71]:
xbases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   O2                 465 non-null    float64
 1   H2O                465 non-null    float64
 2   Trade              455 non-null    object 
 3   POB                111 non-null    object 
 4   Deleted            2 non-null      object 
 5   _nickname          467 non-null    object 
 6   _base              467 non-null    object 
 7   _nickname.1        467 non-null    object 
 8   _Faction           467 non-null    object 
 9   _System            467 non-null    object 
 10  _Region            467 non-null    object 
 11  _Purpose           466 non-null    object 
 12  _Location changed  0 non-null      float64
 13  _Purpose Changed   0 non-null      float64
dtypes: float64(4), object(10)
memory usage: 51.2+ KB


In [68]:
bases[baseiterable]['02'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['O2']


In [76]:
xbases.head()

,O2,H2O,Trade,POB,Deleted,_nickname,_base,_nickname.1,_Faction,_System,_Region,_Purpose,_Location changed,_Purpose Changed
0,3.0,3.0,*,*,NaN,ga03_09_base,Abbeville Production,ga03_09_base,EFL,Picardy,*Gallia,Processing_Metals,NaN,NaN
1,5.0,5.0,*,*,NaN,ku05_02_base,Ainu Depot,ku05_02_base,GC,Hokkaido,Kusari,Outpost_Unlawful,NaN,NaN
2,9.0,9.0,*,NaN,NaN,li08_03_base,Airdrie Hideout,li08_03_base,LH Guard,Alberta,Liberty,Outpost_Unlawful Guard,NaN,NaN
3,1.0,1.0,*,*,NaN,st08_03_base,Akabat,st08_03_base,The Order,Omicron Mu,xtra-House,Population,NaN,NaN
4,3.0,3.0,*,NaN,NaN,ku04_06_base,Akita Border St,ku04_06_base,KSP,Honshu,Kusari,Outpost_Lawful,NaN,NaN


In [78]:
baseiterable = 1
#bases at loc, make new line, mask into xbases to find the correct value for new line, index into value, 0 index into the series the value is saved in. 
bases[baseiterable]['02'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['O2']
bases[baseiterable]['H2O'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['H2O']
bases[baseiterable]['Trade'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['Trade']
bases[baseiterable]['POB'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['POB']
bases[baseiterable]['Deleted'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['Deleted']
bases[baseiterable]['_Faction'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['_Faction']
bases[baseiterable]['_System'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['_System']
bases[baseiterable]['_Region'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['_Region']
bases[baseiterable]['_Purpose'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['_Purpose']
bases[baseiterable]['_Location changed'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['_Location changed']
bases[baseiterable]['_Purpose Changed'] = xbases[xbases['_nickname'] == bases[baseiterable]['base_code']]['_Purpose Changed']

In [80]:
bases[1]

{'base_code': 'Br01_02_base',
 'Name': 'Southampton Shipyard',
 'commodities': [[0.0, -1.0, 0.0, 0.0, 1.0, 1.104, 'commodity_ablativearmor,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.31, 'commodity_aluminium,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.0, 'commodity_ambassadors,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.0, 'commodity_arbiters,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.45, 'commodity_basic_alloys,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.21, 'commodity_beryllium,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.25, 'commodity_beryllium_ca,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.26, 'commodity_bioprocessors,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 1.096, 'commodity_bluediamonds,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.05, 'commodity_bluejillies,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.05, 'commodity_boron,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.17, 'commodity_cobalt,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.77, 'commodity_construction_machinery,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 3.12, 'commodity_consumer_goods_br,'],
  [0.0, -1.0, 0.0, 0.0, 1.0, 2.17, 'commodity_co

In [73]:
xbases[xbases['_nickname'] == 'ga03_09_base']['O2'][0]

3.0

In [81]:
for base in bases[:2]:
    code = base['base_code'].strip('\n').lower()
    xb_entry = xoria_bases[xoria_bases['_nickname'] == code]
    print(xb_entry["_Faction"].values[0])


BPA
BMM


In [60]:
obj.values[0]

'BMM'

In [14]:
class Config(object):
    bases = [] # or load from template

    def __str__(self):
        num_bases = len(self.bases)
        num_commodites = sum([len(comm) for comm in self.bases]) #counts the number of objects inside the bases
        return "this config has {} bases with {} commodies filled".format(str(num_bases), str(num_commodites))

In [15]:
c = Config()

In [16]:
c.bases
pnl = ['metal', 'rocks','gold','FOW','munitions']

In [17]:
c.bases.append(pnl)

In [18]:
print(c)

this config has 1 bases with 5 commodies filled


small economy: Find the 30 bases closest to X

In [48]:
corfu8 = find_nearest_x('Corfu Base', 7, distances)
corfu8.append(base_code_lookup['Corfu Base'].lower())

In [49]:
c30_coms = ['Food', 
            'water', 
            'fuel', 
            'air', 
            'o2', 
            'guns',
            'metal',
            'rocks',
            'batteries',
            'cattle',
            'electrics',
            'scrap',
            'toxins',
            'plastics',
            'toys',
            ]
c8_coms = ['toxins', 'toys', 'metal','scrap','electrics', 'rocks','air','water','batteries','guns','fuel','o2','cattle']

In [50]:
dist =distances[distances['end'].isin(corfu8)]
distc8 = dist[dist['start'].isin(corfu8)]

config =[[base,[produced commodity], [resold commodity], [bought commodity]]]

In [51]:
[base_name_lookup[code] for code in corfu8]

['Corfu Base']

In [52]:
config = [['ew12_04_base', ['toxins', 'toys'],[],['cattle','o2','metal','scrap','electrics','air']],
        ['ew04_01_base',['scrap', 'electrics', 'rocks'], [],['cattle','air', 'water', 'metal', 'batteries','air']],
        ['ga09_01_base',['guns', 'air'],['toxins'],['cattle','o2','water','fuel','metal', 'batteries','electrics']],
        ['bw06_01_base',['cattle','batteries', 'o2'],['air'],['metal','toxins','air']],
        ['ga01_16_base',['fuel','metal'],['air'],['cattle','toxins','electrics','rocks','o2']],
        ['ga14_01_base',['water','electrics','o2'],['air'],['cattle','rocks', 'batteries', 'toxins']],
        ['ew06_02_base',['cattle', 'rocks','scrap'],[],['toxins','o2','water','air', 'toys', 'electrics','air','metal']],
        ['bw12_02_base',['fuel','air','metal'],[],['water', 'o2','cattle']],
        ['ew07_02_base',['water','guns'],['o2'],['fuel','electrics', 'rocks','air','metal']],
        #['hi18_04_base',['metal','scrap'],[],['fuel','rocks', 'Food']],
        #['bw06_02_base',['rocks'],[],['air', 'water']],
        #['ku08_01_base',['electrics'],[],['water','guns','batteries']],
        #['hi02_01_base',['scrap'],[],['toys','water','metal']],
        #['bw12_05_base',['toxins'],['toys'],['plastics','batteries']],
        #['hi18_03_base',['plastics'],[],['toxins','fuel','air','water']],
        #['ga08_09_base',['water','Food'],[],['rocks']],
        #['ga03_10_base',['fuel'],[],['cattle','scrap','guns']],
        #['ga05_11_base',['air'],[],['guns','plastics']],
        #['ew06_05_base',['toys'],[],['batteries','rocks','scrap']],
        #['hi02_02_base',['cattle'],[],['scrap','guns','fuel','water']],
        #['ew06_01_base',['toys'],[],['batteries','cattle','scrap', 'Food']],
        #['bw05_03_base',['o2'],[],['toys','guns','fuel', 'Food']],
        #['bw69_03_base',['guns', 'Food'],[],['toxins','metal','plastics','cattle']],
        #['hi02_03_base',['o2'],[],['air','plastics']],
        #['bw07_01_base',['air'],[],['toys', 'water', 'Food']],
        #['ew06_03_base',['guns'],[],['scrap','metal','rocks']],
        #['bw01_05_base',['metal'],[],['scrap','electrics','cattle']],
        #['hi20_02_base',['rocks'],[],['metal','batteries','toys']],
        #['rh10_02_base',['cattle'],['scrap'],['batteries','fuel','o2']],
        ['ew04_02_base',['batteries'],[],['water','toxins','guns', 'scrap', 'metal','electrics','air']]]
         

In [98]:
market =gen_market_from_config(config, distances)

++++++++++======================================================================+++++++++++

In [115]:
market['ga01_16_base']

('metal', 'sells', 1)

In [152]:
file_path ='test_market.ini'
def write_ini(file_path, market, config):
    '''
    function that takes the market dictonary and formats it for freelancer, and saves it as a text file
    ++++++++++
    Parameters
    file_path (str): name and path of file
    market (defaultdict(list)): per base formatting of all commodities
    ++++++++++
    Returns:
    none, 
    writes a file
    '''
    lines = []
    for item in market.keys():

        lines.append('[BaseGood]')
        lines.append('base = '+ item)
        lines.append(';'+base_name_lookup[item])

        for commodity in market[item]:
            if commodity[1] == 'sells':
                function_tag = '1'
            else:
                function_tag = '0'
            price_mult = str(commodity[2]) # For a working version, this needs to reference something in config to set prices
            mgood =['Marketgood =',commodity[0],'0, -1, 0, 0,', function_tag, str(commodity[2])]
            lines.append(' '.join(mgood))

    with open(file_path, 'w') as f:
        for line in lines:
            f.write(line+'\n')


per base that is buying
Marketgood = commodity_ablativearmor, 0, -1, 0, 0, 1, 5.778000

In [153]:
write_ini(file_path,market, config)

In [ ]:

with open('base_code_lookup.csv', 'w') as f:
    for key in base_code_lookup.keys():
        f.write("%s, %s\n" % (key, base_code_lookup[key]))

In [ ]:
sortd, df =sort_by_closest_base(['st39_04_base', 'br01_04_base'], distances)

In [ ]:
len(sortd[0])

In [ ]:
sortd.
#for i in list_of_bases:
    

In [ ]:
df

In [1]:
bases

NameError: name 'bases' is not defined